In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ollama
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_community.llms import Ollama
import textwrap

In [16]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

def printText(text):
    wrapped_text = textwrap.fill(text, width=100) #text is the object which you want to print
    print(wrapped_text)

llm = Ollama(
    base_url='http://localhost:11434',
    model="llama3"
)

In [17]:
from langchain.embeddings import OllamaEmbeddings
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

In [18]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = oembed
).from_loaders([loader])

In [19]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [20]:
response = index.query(query, llm = llm)

In [21]:
display(Markdown(response))

Here is the list of shirts with sun protection in a table format using Markdown:

| Shirt Name | Description | Sun Protection Rating |
| --- | --- | --- |
| **Sun Shield Shirt** | Block the sun, not the fun - high-performance sun shirt provides guaranteed UV ray protection. Wicks moisture for quick-drying comfort and abrasion resistant for season after season of wear. | SPF 50+ (98% blocking) |
| **Sunrise Tee** | Lightweight, high-performance fabric wicks away moisture and dries quickly. Built-in UPF 50+ rated sun protection with wrinkle-free fabric. Originally designed for fishing or travel. | UPF 50+ |

Let me know if you have any further questions!

## Step by Step

In [22]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [23]:
docs = loader.load()
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [24]:
embed = oembed.embed_query("Hi my name is Harrison")

In [25]:
print(len(embed))

768


In [26]:
print(embed[:5])

[0.49577486515045166, 0.3908696174621582, -2.179100513458252, -0.5123597383499146, -0.13692554831504822]


In [27]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    oembed
)

In [28]:
query = "Please suggest a shirt with sunblocking"

In [29]:
docs = db.similarity_search(query)

In [30]:
len(docs)

4

In [31]:
retriever = db.as_retriever()

In [32]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [33]:
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [34]:
display(Markdown(response))

Here is the list of shirts with sun protection:

| Shirt Name | Description | Sun Protection |
| --- | --- | --- |
| Sunrise Tee | Lightweight, high-performance fabric wicks away moisture and dries quickly. Built-in UPF 50+ rated. | UPF 50+ |
| Sun Shield Shirt by | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. | UPF 50+, SPF 50+ |
| Men's Stay Grounded Tee, Short-Sleeve | Permethrin-treated for insect repellency and UPF 50+ rated. | UPF 50+ |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve shirt with wrinkle-resistant and moisture-wicking fabric, UPF 50+ rated. | UPF 50+ |

Here's a brief summary of each shirt:

* **Sunrise Tee**: A lightweight, high-performance button-down shirt designed for hot weather, featuring built-in UPF 50+ protection.
* **Sun Shield Shirt by**: A high-performance sun shirt with SPF 50+ protection, blocking 98% of the sun's harmful rays. It's recommended by The Skin Cancer Foundation as an effective UV protectant.
* **Men's Stay Grounded Tee, Short-Sleeve**: A short-sleeve tee treated with permethrin for insect repellency and featuring UPF 50+ rated protection.
* **Tropical Breeze Shirt**: A lightweight, breathable long-sleeve shirt designed for hot weather, featuring wrinkle-resistant and moisture-wicking fabric, as well as UPF 50+ rated protection.

In [35]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [36]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."
response = qa_stuff.run(query)
display(Markdown(response))

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


Here is the list of shirts with sun protection:

| Shirt Name | Description | Sun Protection |
| --- | --- | --- |
| Sunrise Tee | Lightweight, high-performance fabric wicks away moisture and dries quickly. | UPF 50+ rated |
| Sun Shield Shirt by [Brand] | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. | UPF 50+ rated |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve shirt with wrinkle-resistant and moisture-wicking fabric. | UPF 50+ rated |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with a fit to flatter and SunSmart™ protection to block the sun's harmful UV rays. | UPF 50+ rated |

Each of these shirts provides high-level sun protection with a UPF 50+ rating, blocking 98% of the sun's harmful UV rays. They also offer features such as moisture-wicking fabric, wrinkle-resistance, and quick-drying capabilities to keep you comfortable while outdoors.

In [37]:
response = index.query(query, llm=llm)
response

"Here is the list of shirts with sun protection:\n\n| Shirt Name | Description | Sun Protection |\n| --- | --- | --- |\n| Sun Shield Shirt | Block the sun, not the fun - our high-performance sun shirt is guaranteed to protect from harmful UV rays. | SPF 50+ |\n| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly. | UPF 50+ |\n\nNote: Both shirts have built-in sun protection with a rating of either SPF 50+ or UPF 50+, which is the highest rated sun protection possible."

In [38]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=oembed,
).from_loaders([loader])